In [2]:
!pip install googlemaps yelpapi pandas scikit-learn

import googlemaps
import requests
from yelpapi import YelpAPI
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.exceptions import NotFittedError
from datetime import datetime

# Install necessary libraries
# Uncomment if running locally
# !pip install googlemaps yelpapi pandas scikit-learn

# --- Configuration ---
GOOGLE_MAPS_API_KEY = "http://maps.app.goo.gl/AbLBny2kZBek65Xm7"
YELP_API_KEY = "YOUR_YELP_API_KEY"
OPENWEATHER_API_KEY = "http://openweathermap.org/current"
VILLAGE_LOCATION = "Village The Soul of India, Hicksville"

# --- Helper Functions ---
def get_weather(api_key, lat, lon):
    "Fetch weather data from OpenWeather API."
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=imperial"
    response = requests.get(url)
    if response.status_code == 200:
      return response.json()
    else:
        print("Error fetching weather data:", response.status_code, response.text)
        return None

def should_use_predicted_price(predicted_price, temperature, rain_indicator, busy_indicator):
    return predicted_price != "Unavailable" and (temperature < 45 or rain_indicator == 1 or busy_indicator == 1)

def determine_busy_indicator():
    """Determine if the location is busy based on time and day."""
    current_hour = datetime.now().hour
    current_day = datetime.now().weekday()  # Monday = 0, Sunday = 6

    # Example logic: peak hours during weekdays (11 AM to 1 PM, 6 PM to 8 PM) and weekends (5 PM to 9 PM)
    if current_day < 5:  # Weekdays
        return 1 if (11 <= current_hour <= 13 or 18 <= current_hour <= 20) else 0
    else:  # Weekends
        return 1 if (17 <= current_hour <= 21) else 0

# --- Main Script ---

# 1. Initialize API Clients
try:
    gmaps = googlemaps.Client(key="http://maps.app.goo.gl/AbLBny2kZBek65Xm7")

except ValueError as e:
    print("Error initializing Google Maps client. Check your API key.", e)
    exit()

yelp_api = YelpAPI(YELP_API_KEY)

# 2. Fetch Restaurant Data from Yelp
try:
    village_yelp = yelp_api.business_query(id="village-the-soul-of-india-hicksville")
    print("Village Data from Yelp:", village_yelp)
except Exception as e:
    print("Error fetching from Yelp API:", e)

# Placeholder for nearby restaurants
nearby_restaurants = [
    {"name": "Restaurant A", "menu_items": [
        {"name": "Dish 1", "price": 10},
        {"name": "Dish 2", "price": 12}
    ], "address": "address1"},
    {"name": "Restaurant B", "menu_items": [
        {"name": "Dish 3", "price": 9},
        {"name": "Dish 4", "price": 11}
    ], "address": "address2"}
]

# 3. Fetch Weather and Geocode Data
try:
    geocode_result = gmaps.geocode(VILLAGE_LOCATION)
    if geocode_result:
        lat = geocode_result[0]['geometry']['location']['lat']
        lon = geocode_result[0]['geometry']['location']['lng']
        weather_data = get_weather(OPENWEATHER_API_KEY, lat, lon)

        if weather_data:
            temperature = weather_data['main']['temp']
            description = weather_data['weather'][0]['description']
        else:
            temperature = None
            description = "Unavailable"
    else:
        print("Geocoding failed.")
        temperature = None
        description = "Unavailable"
except Exception as e:
    print("Error during geocoding or weather fetching:", e)
    temperature = None
    description = "Unavailable"

# 4. Prepare Data for Machine Learning Model
# Replace placeholder data with real application data
real_data = {
    'Price': [20, 25, 30, 35],
    'Temperature': [45, 55, 65, 75],
    'Rain': [0, 1, 0, 0],
    'Busy': [0, 1, 1, 0]
}
df = pd.DataFrame(real_data)

# Train Model
X = df[['Temperature', 'Rain', 'Busy']]
y = df['Price']
model = LinearRegression()
model.fit(X, y)

# 5. Make Predictions Based on Current Weather
if temperature is not None:
    rain_indicator = 1 if "rain" in description.lower() or "snow" in description.lower() else 0
    busy_indicator = determine_busy_indicator()
    new_data = pd.DataFrame({'Temperature': [temperature], 'Rain': [rain_indicator], 'Busy': [busy_indicator]})
    try:
        predicted_price = model.predict(new_data)[0]
    except ValueError as e:
        print("Error during prediction. Ensure input data is valid:", e)
        predicted_price = "Unavailable"
else:
    predicted_price = "Unavailable"
    rain_indicator = 0  # or any default value you prefer
    busy_indicator = determine_busy_indicator()

# 6. Display Results
print("\nVillage Menu (with predicted prices):")
menu = [{"name": "Dish 1", "lowest_price": 12}, {"name": "Dish 2", "lowest_price": 15}]

for item in menu:
    if should_use_predicted_price(predicted_price, temperature, rain_indicator, busy_indicator):
        price_to_display = predicted_price
    else:
        price_to_display = item["lowest_price"]
    print(f"- {item['name']} Price: {price_to_display}")

print("\nNearby Restaurants:")
for restaurant in nearby_restaurants:
    print(f"\nRestaurant: {restaurant['name']}")
    for item in restaurant["menu_items"]:
        print(f"- {item['name']} Price: {item['price']}")


Error initializing Google Maps client. Check your API key. Invalid API key provided.
Error fetching from Yelp API: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/village-the-soul-of-india-hicksville
Error during geocoding or weather fetching: name 'gmaps' is not defined

Village Menu (with predicted prices):
- Dish 1 Price: 12
- Dish 2 Price: 15

Nearby Restaurants:

Restaurant: Restaurant A
- Dish 1 Price: 10
- Dish 2 Price: 12

Restaurant: Restaurant B
- Dish 3 Price: 9
- Dish 4 Price: 11
